In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [2]:
df=pd.read_csv("/kaggle/input/emotional-monitoring-dataset/emotional_monitoring_dataset_with_target.csv")

In [3]:
df

,HeartRate,SkinConductance,EEG,Temperature,PupilDiameter,SmileIntensity,FrownIntensity,CortisolLevel,ActivityLevel,AmbientNoiseLevel,LightingLevel,EmotionalState,CognitiveState,EngagementLevel
0,61,8.937204,11.794946,36.501723,3.330181,0.689238,0.189024,0.603035,136,59,394,engaged,distracted,3
1,60,12.635397,19.151412,36.618910,3.428995,0.561056,0.091367,0.566671,155,39,479,engaged,distracted,1
2,81,3.660028,6.226098,36.176898,2.819286,0.417951,0.227355,1.422475,55,30,832,partially engaged,focused,3
3,119,0.563070,4.542968,37.205293,2.192961,0.140186,0.502965,1.669045,39,40,602,disengaged,focused,3
4,118,0.477378,0.996209,37.248118,2.450139,0.064471,0.695604,1.854076,10,42,908,disengaged,focused,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,98,3.897648,7.681519,36.274526,2.624275,0.404309,0.204719,1.215872,65,50,913,partially engaged,distracted,2
996,109,0.439062,0.352790,37.173929,2.489483,0.070776,0.638161,1.826544,23,43,642,disengaged,distracted,2
997,108,1.077287,1.836462,37.073454,2.370298,0.011001,0.595518,1.781096,8,43,620,disengaged,distracted,2
998,76,14.260010,19.309704,36.708047,3.393744,0.653693,0.171151,0.783958,110,38,779,engaged,distracted,1


In [4]:
X=df.drop(columns=["CognitiveState"])
# Y=df["CognitiveState"]

In [5]:
X=pd.get_dummies(X)
label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(df["CognitiveState"])

In [6]:
# Y

In [7]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [8]:
for i in [RandomForestClassifier(),LogisticRegression(),GradientBoostingClassifier(),AdaBoostClassifier(),KNeighborsClassifier(),BernoulliNB()]:
    random_classif=i
    random_classif.fit(X_train,Y_train)
    print(f"The Accuracy of{i} ",random_classif.score(X_test,Y_test))

The Accuracy ofRandomForestClassifier()  0.495
The Accuracy ofLogisticRegression()  0.51


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


The Accuracy ofGradientBoostingClassifier()  0.43
The Accuracy ofAdaBoostClassifier()  0.47
The Accuracy ofKNeighborsClassifier()  0.485
The Accuracy ofBernoulliNB()  0.56


# Quantum Deep learning

In [9]:
!pip install cirq

In [10]:
import cirq
import tensorflow as tf
from keras.layers import Layer,Dense,Flatten,BatchNormalization,Dropout
from keras.models import Sequential
from keras.initializers import HeNormal,GlorotUniform
from keras.regularizers import l2,l1_l2
from keras.callbacks import EarlyStopping

In [11]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(14)]
    circuit=cirq.Circuit()
    for i,value  in enumerate(data):
        q=i%len(qubit)
        circuit.append(cirq.H(qubit[q]))
        circuit.append(cirq.rx(value*np.pi).on(qubit[q]))
        circuit.append(cirq.X(qubit[q]))
        circuit.append(cirq.ry(value*np.pi).on(qubit[q]))
        circuit.append(cirq.rz(value*np.pi).on(qubit[q]))
        if (i+1)%len(qubit)<len(qubit):
            circuit.append(cirq.CNOT(qubit[q],qubit[(i+1)%len(qubit)]))
            circuit.append(cirq.measure(qubit[q],key=f"qubit_{i}"))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    simulator=cirq.Simulator()
    for inputs_data in data:
        circuit,qubit=quantum_circuit(inputs_data)
        result=simulator.run(circuit)
        measure0=[]
        for i in range(len(inputs_data)):
            measure0.append(result.measurements[f"qubit_{i}"].flatten()[0])
        results.append(measure0)
    return np.array(results,dtype=np.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        outputs=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
        outputs.set_shape(inputs.shape)
        return outputs
model=Sequential([
    QuantumLayer(),
    Dense(32,activation="relu",kernel_initializer="glorot_uniform",kernel_regularizer=l2(0.01)),
    Dense(1,activation="sigmoid")
])
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [12]:
model.fit(X_train,Y_train,epochs=10,batch_size=32)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 929ms/step - accuracy: 0.4790 - loss: 0.9444
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 934ms/step - accuracy: 0.4907 - loss: 0.8881
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 947ms/step - accuracy: 0.5051 - loss: 0.8627
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 933ms/step - accuracy: 0.5193 - loss: 0.8452
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 941ms/step - accuracy: 0.5496 - loss: 0.8250
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 943ms/step - accuracy: 0.4497 - loss: 0.8213
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 928ms/step - accuracy: 0.5501 - loss: 0.7981
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 937ms/step - accuracy: 0.4975 - loss: 0.7934
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 939ms/step - accuracy: 0.5259 - loss: 0.7807
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 937ms/step - accuracy: 0.4924 - loss: 0.7726


In [13]:
loss,accuracy=model.evaluate(X_test,Y_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 809ms/step - accuracy: 0.5419 - loss: 0.7644


In [16]:
model.predict(X_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 845ms/step


array([[0.4069045 ],
       [0.5030354 ],
       [0.48009232],
       [0.43343174],
       [0.48869377],
       [0.4853055 ],
       [0.49544615],
       [0.53068763],
       [0.49652323],
       [0.4806882 ],
       [0.50585574],
       [0.50447375],
       [0.51094556],
       [0.46178746],
       [0.4728657 ],
       [0.5141493 ],
       [0.47736448],
       [0.4999761 ],
       [0.545433  ],
       [0.4701733 ],
       [0.5415765 ],
       [0.52488524],
       [0.48572373],
       [0.5204854 ],
       [0.49009258],
       [0.47222558],
       [0.5239726 ],
       [0.47879097],
       [0.5203051 ],
       [0.49530044],
       [0.4626554 ],
       [0.5229758 ],
       [0.46904698],
       [0.48445195],
       [0.47964495],
       [0.5151268 ],
       [0.4972921 ],
       [0.49972242],
       [0.49480784],
       [0.46620134],
       [0.50265336],
       [0.4706224 ],
       [0.44106415],
       [0.46406478],
       [0.45340487],
       [0.47127166],
       [0.487681  ],
       [0.503

In [14]:
model1=Sequential([
    # QuantumLayer(),
    Dense(32,activation="relu",kernel_initializer="glorot_uniform",kernel_regularizer=l2(0.01)),

    Dense(1,activation="sigmoid")
])
model1.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model1.fit(X_train,Y_train,epochs=10,batch_size=32)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5224 - loss: 24.1072   
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5372 - loss: 6.2982 
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5076 - loss: 4.5362 
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5202 - loss: 2.8360 
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5573 - loss: 1.6834 
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5343 - loss: 1.2081 
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4702 - loss: 1.2348 
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4956 - loss: 1.1226 
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4790 - loss: 1.2692 
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5150 - loss: 1.0678 


In [15]:
loss,accuracy=model1.evaluate(X_test,Y_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4520 - loss: 1.1569  


In [17]:
model1.predict(X_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


array([[0.73750377],
       [0.78132397],
       [0.92011535],
       [0.22684638],
       [0.7411692 ],
       [0.27339384],
       [0.42000455],
       [0.34579685],
       [0.573904  ],
       [0.67323685],
       [0.4278671 ],
       [0.5354826 ],
       [0.5129726 ],
       [0.7536379 ],
       [0.8350036 ],
       [0.8814531 ],
       [0.07598606],
       [0.70823526],
       [0.2459328 ],
       [0.7118363 ],
       [0.70896703],
       [0.46362427],
       [0.7383949 ],
       [0.5290598 ],
       [0.5763919 ],
       [0.79168385],
       [0.84439284],
       [0.78541   ],
       [0.7148954 ],
       [0.78031856],
       [0.74702513],
       [0.1025986 ],
       [0.9355961 ],
       [0.72815657],
       [0.7138041 ],
       [0.63791394],
       [0.840574  ],
       [0.5825428 ],
       [0.853905  ],
       [0.54334205],
       [0.8011262 ],
       [0.87076896],
       [0.9768716 ],
       [0.78712684],
       [0.6571    ],
       [0.92693293],
       [0.9202605 ],
       [0.091